In [1]:
#@title
%matplotlib inline
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); 
sns.set_style("white"); sns.set_style("ticks"); sns.set_context("paper")
from pylab import rcParams; rcParams['figure.figsize'] = 5, 5
from constants import MAX_SEQUENCE_LENGTH_LIST, NB_CLASSES_LIST
from keras_utils import evaluate_model, set_trainable, visualize_context_vector, visualize_cam
import argparse
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, PReLU, Dense, LSTM, multiply, concatenate, Activation
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from keras_utils import train_model
from layer_utils import AttentionLSTM
from keras.optimizers import Adam

prefix = 'SW13'
datasetindex = 24
DATASET_INDEX = datasetindex
MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH_LIST[DATASET_INDEX]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX]
TRAINABLE = True

model_num = 3

def generate_model_3():
    ip = Input(shape=(1, MAX_SEQUENCE_LENGTH))
    x = AttentionLSTM(8)(ip) # 8 # 32
    x = Dropout(0.8)(x)
    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = GlobalAveragePooling1D()(y)
    x = concatenate([x, y])
    out = Dense(NB_CLASS, activation='softmax')(x)
    model = Model(ip, out)

    return model

model = generate_model_3()

optm = Adam(lr=1e-3)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('{prefix}_{model_num}_weights.h5'.format(prefix=prefix, model_num=model_num))
read_dictionary = np.load('{prefix}_labels_dict.npy'.format(prefix=prefix)).item()
read_dictionary = {v: k for k, v in read_dictionary.items()}

/home/snd2/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [16]:
#####################################
###### Put filename here ############
 
sample_name = '../../unedited_data/Dave_data/6.txt'

###### including .csv/.txt ########## 
#####################################

aa = pd.read_csv('{sample_name}'.format(sample_name=sample_name), skiprows=20, header=None)
# aa = aa.iloc[0]
aa = np.array(aa[2])
aa = aa.reshape(1, 1, MAX_SEQUENCE_LENGTH)
aa_df = pd.DataFrame(np.transpose(model.predict(aa)))
aa_df['Label'] = [read_dictionary[int(x)] for x in aa_df.index]
aa_df.columns.values[0] = 'Probability' 
aa_df.set_index('Label', inplace=True)

print(aa_df.sort_values(by=['Probability'], ascending=False).round(3))

          Probability
Label                
PQ                1.0
DQ                0.0
Pb                0.0
MeP               0.0
Hg                0.0
seawater          0.0
NP                0.0
Cd                0.0
Cu                0.0
BPA               0.0
13-DC             0.0
Ag                0.0
1-Dod             0.0


In [ ]:
#@title
from keras import backend as K

def build_function(model, layer_names=None, outputs=None):
    inp = model.input

    if layer_names is not None and (type(layer_names) != list and type(layer_names) != tuple):
        layer_names = [layer_names]

    if outputs is None:
        if layer_names is None:
            outputs = [layer.output for layer in model.layers] # all layer outputs
        else:
            outputs = [layer.output for layer in model.layers if layer.name in layer_names]
    else:
        outputs = outputs

    funcs = [K.function([inp] + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions
    return funcs

def get_outputs(model, inputs, eval_functions, verbose=False):
    if verbose: print('----- activations -----')
    outputs = []
    layer_outputs = [func([inputs, 1.])[0] for func in eval_functions]
    for layer_activations in layer_outputs:
        outputs.append(layer_activations)
    return outputs

In [ ]:
#@title
class_weights = model.layers[-1].get_weights()[0]

conv_layers = [layer for layer in model.layers if layer.__class__.__name__ == 'Conv1D']

final_conv = conv_layers[-1].name
final_softmax = model.layers[-1].name
out_names = [final_conv, final_softmax]

class_id = 0

if class_id > 0:
    class_id = class_id - 1

# y_train_ids = np.where(y_train[:, 0] == class_id) # class_id
y_train_ids = 'test'
sequence_input = aa
choice = np.random.choice(range(len(sequence_input)), 1)
sequence_input = sequence_input[choice, :, :]

eval_functions = build_function(model, out_names)
conv_out, predictions = get_outputs(model, sequence_input, eval_functions)

conv_out = conv_out[0, :, :]
conv_out = conv_out.transpose((1, 0))
conv_channels = conv_out.shape[0]

sequence_length = MAX_SEQUENCE_LENGTH

conv_cam = np.zeros(sequence_length, dtype=np.float32)
for i, w in enumerate(class_weights[conv_channels:, class_id]): # class_id
    conv_cam += w * conv_out[i, :]

conv_cam /= conv_cam.max()

sequence_input = sequence_input.reshape((-1, 1))
conv_cam = conv_cam.reshape((-1, 1))

sequence_df = pd.DataFrame(sequence_input,
                           index=range(sequence_input.shape[0]),
                           columns=range(sequence_input.shape[1]))

conv_cam_df = pd.DataFrame(conv_cam,
                           index=range(conv_cam.shape[0]),
                           columns=[1])

fig, axs = plt.subplots(2, 1, squeeze=False,
                        figsize=(6, 6))
sequence_df.plot(title='Data',
                 color='black',
                 linewidth=2, 
                 subplots=False,
                 legend=None,
                 ax=axs[0][0])
conv_cam_df.plot(title='CAM',
                 color='black',
                 linewidth=2,               
                 subplots=False,
                 legend=None,
                 ax=axs[1][0])

plt.show()

# ax1 = sns.heatmap(np.transpose(conv_cam_df), cbar=False) #, cmap="RdYlGn")
# ax2 = ax1.twiny()
# ax2.plot(sequence_df, color='black', linewidth=2)
# ax1.set_ylim(-20, 20)